## Load data and packages

In [9]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
os.chdir('D:/Grad Study!/Geeky things (Competition/Youtube data/youtube-new')
ca = pd.read_csv('./CAvideos.csv',encoding='utf-8')
de = pd.read_csv('./DEvideos.csv',encoding='utf-8')
fr = pd.read_csv('./FRvideos.csv',encoding='utf-8')
uk = pd.read_csv('./GBvideos.csv',encoding='utf-8')
us = pd.read_csv('./USvideos.csv',encoding='utf-8')

In [3]:
# Add a column to indicate geo location, then append all tables to one
ca['country'] = 'CA'
de['country'] = 'DE'
fr['country'] = 'FR'
uk['country'] = 'UK'
us['country'] = 'US'

df = pd.concat([ca,de,fr,uk,us], ignore_index=True)

In [4]:
# Check missing values
df.isnull().sum()

video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        1
likes                        1
dislikes                     1
comment_count                2
thumbnail_link               2
comments_disabled            2
ratings_disabled             2
video_error_or_removed       2
description               4099
country                      0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116739 entries, 0 to 116738
Data columns (total 17 columns):
video_id                  116739 non-null object
trending_date             116739 non-null object
title                     116739 non-null object
channel_title             116739 non-null object
category_id               116739 non-null int64
publish_time              116739 non-null object
tags                      116739 non-null object
views                     116738 non-null object
likes                     116738 non-null object
dislikes                  116738 non-null object
comment_count             116737 non-null float64
thumbnail_link            116737 non-null object
comments_disabled         116737 non-null object
ratings_disabled          116737 non-null object
video_error_or_removed    116737 non-null object
description               112640 non-null object
country                   116739 non-null object
dtypes: float64(1), int64(1), object(15)
memory usage: 1

In [6]:
# Drop rows where views, likes and comments == missing
df = df[np.isfinite(df['comment_count'])]

In [7]:
# Set trending date to datetime
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m')
df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# New column on date difference between trending date and publish date
df['date_diff'] = df['trending_date'] - df['publish_time']

In [8]:
# Convert views, likes and dislikes to int64
int_list = ['views', 'likes', 'dislikes', 'comment_count']
for column in int_list:
    df[column] = df[column].astype(int)

str_list = ['category_id']
for column in str_list:
    df[column] = df[column].astype(str)

In [34]:
#Map category id to category title, and map it back to the dataset
id_category = {}

with open('CA_category_id.json', 'r') as f:
    data = json.load(f)
    for category in data['items']:
        id_category[category['id']] = category['snippet']['title']

#Map category title to the dataset
df.insert(4, 'category', df['category_id'].map(id_category))
df = df.drop('category_id',axis=1)

In [37]:
#export the dataset to csv
df.to_csv('video_stats.csv',encoding='utf-8')